In [ ]:
# Property Correlation Plotter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from mp_api.client import MPRester


# Set your Materials Project API Key
API_KEY = "Put My API Key Here" # Replace with the API Key

def property_correlation_plotter(chemsys="Al-N"):
    """
    This code shows the correlation between formation energy and band gap for a specific material/chemical system (in this case I decided to use Al-N)

    Args:
        chemsys (str): Chemical system (e.g., "Al-N")
    """
    print(f"\nAnalyzing property correlation for {chemsys} system...")
    print(f"Plotting: Band Gap vs. Formation Energy")
    
    # Connect to Materials Project
    with MPRester(API_KEY) as mpr:
        docs = mpr.materials.summary.search(
            chemsys=chemsys,
            fields=[
                "material_id", 
                "formula_pretty", 
                "structure", 
                "band_gap", 
                "density", 
                "energy_above_hull", 
                "formation_energy_per_atom",
                "volume"
            ]
        )
        
        # Build DataFrame
        data = []
        for doc in docs:
            data.append({
                "material_id": doc.material_id,
                "formula": doc.formula_pretty,
                "band_gap": doc.band_gap,
                "density": doc.density,
                "stability": doc.energy_above_hull,
                "formation_energy": doc.formation_energy_per_atom,
                "volume": doc.volume
            })
        df = pd.DataFrame(data)
        
        print(f"Retrieved {len(df)} materials.")
        
        # Drop rows with missing data
        df = df[["formation_energy", "band_gap", "formula", "material_id"]].dropna()
        
        # Calculate correlation coefficient
        corr = df["formation_energy"].corr(df["band_gap"])
        print(f"\nCorrelation coefficient between formation energy and band gap: {corr:.4f}")
        
        # Matplotlib plot
        plt.figure(figsize=(10, 6))
        plt.scatter(df["formation_energy"], df["band_gap"], alpha=0.7)
        plt.xlabel("Formation Energy (eV/atom)")
        plt.ylabel("Band Gap (eV)")
        plt.title(f"Band Gap vs. Formation Energy for {chemsys} Materials")
        plt.grid(True, alpha=0.3)
        plt.savefig(f"{chemsys}_bandgap_vs_formation_energy.png")
        plt.show()
        
        # Plotly interactive plot
        fig = px.scatter(
            df,
            x="formation_energy",
            y="band_gap",
            hover_name="formula",
            hover_data=["material_id"],
            title=f"Band Gap vs. Formation Energy for {chemsys} Materials",
            labels={
                "formation_energy": "Formation Energy (eV/atom)",
                "band_gap": "Band Gap (eV)"
            }
        )
        fig.update_layout(template="plotly_white")
        fig.show()
        
        return df

def main():
  
    print("Property Correlation Plotter")
   
    
    # The chemical system that is used:
    chemical_system = "Al-N"
    
    df = property_correlation_plotter(chemical_system)
    print("\nPlotting complete!")

if __name__ == "__main__":
    main()